In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets

import seaborn as sns

In [ ]:
%matplotlib inline
sklearn.__version__ 

# Linear Regression with Numpy (only)

### Let's read the file containing our data
Pandas provides an easier way to read the data and convert to numpy

In [ ]:
data_df = sklearn.datasets.load_diabetes(as_frame=True).data
data_df.head()

In [ ]:
data_arr = data_df.values
data_arr

In [ ]:
data_arr.shape

#### Extract X, y

In [ ]:
X_raw = data_arr[:,1:] # get everything but the first column
y_raw = data_arr[:, :1]# get the first column (I'm not just doing data_arr[:,-1] to avoid (x,) dimensions)

In [ ]:
X_raw.shape, y_raw.shape

#### Normalize data

In [ ]:
#X_raw = X_raw / X_raw.sum()
#y_raw = y_raw / y_raw.sum()

#### Add bias to data

In [ ]:
X = np.hstack((np.ones(y_raw.shape),X_raw))
y = y_raw

In [ ]:
X.shape, y.shape

In [ ]:
X.dtype, y.dtype

#### Use gradient descent to solve for coefficients

In [ ]:
learning_rate = 0.001
iterations = 100_000

features = X.shape[1]

In [ ]:
w = np.ones((features,1))

for i in range(iterations):
    w -= (learning_rate) * (2) * X.T.dot(X.dot(w) - y)

In [ ]:
w.flatten()

##### MSE

In [ ]:
observations = X.shape[0]
np.sum((X.dot(w) - y)**2)/ (observations)

#### Try again, but do a grid search for hyper parameters

In [ ]:
learning_rate_list = [0.001, 0.0001, 0.00001]
iteration_list = [100, 1_000, 10_000, 100_000]

hyper_parameters = list()

for learning_rate in learning_rate_list:
    print(f"Trying learning rate {learning_rate} and batch size {iterations}")
    for iterations in iteration_list:
        w = np.ones((features,1))
        for i in range(iterations):
            w -= (learning_rate) * (2) * X.T.dot(X.dot(w) - y)
            error = np.sum(X.dot(w) - y) # not squared to avoid overflow error
        hyper_parameters.append((learning_rate, iterations, error))

#### Cleaner version of the same code

In [ ]:
def fit(X, y, iterations=10000, learning_rate=0.0001, callback=None, callback_freq=100_000):

    observations = X.shape[0]
    features = X.shape[1]

    w = np.ones((features,1))

    predictor = lambda X: X.dot(w)
    mse       = lambda X, y: np.sum((predictor(X) - y)**2)/observations

    errors = np.zeros(iterations)
    for i in range(iterations): 
        prediction = predictor(X)
        error = prediction - y
        errors[i] = mse(X, y)
        if callback and i % callback_freq == 0: callback(i, w, errors[i])
        #gradient = (2/observations) * X.T.dot(error)
        gradient =  2 * X.T.dot(error)
        w -= learning_rate * gradient
    
    return predictor, w, mse, errors

In [ ]:
%%time 

learning_rate_list = [0.001, 0.0001]
iteration_list = [10, 100, 1_000, 10_000, 100_000]

callback = lambda i, w, e:print(e)

hyper_parameters = list()

for learning_rate in learning_rate_list:
    for iterations in iteration_list:
        
        print(f"Trying learning rate {learning_rate} and batch size {iterations}")
        
        predictor, w, mse, errors = fit(X, y, iterations = iterations, learning_rate=learning_rate, callback=None)
        hyper_parameters.append((learning_rate, iterations, mse(X, y), errors))

In [ ]:
#hyper_parameters

In [ ]:
pd.DataFrame({'a':1, 'b':2, 'c':[1,2,3]})

In [ ]:
dfs = list()
for hp in hyper_parameters:
    lr, epoch, mse, errors = hp
    error_indexes = list(range(len(errors)))
    dfs.append(pd.DataFrame({'lr':lr, 'epoch':epoch, 'mse':mse, 'error':errors, 'error_indexes':error_indexes}))

hyper_parameters_df = pd.concat(dfs)

In [ ]:
hyper_parameters_df

In [ ]:
sns.relplot(x='error_indexes'
            , y='error'
            , col='lr'
            , row='epoch'
            , data=hyper_parameters_df
            , kind='line'
            , facet_kws={'sharey':False, 'sharex':False})

### Check work

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%time reg = LinearRegression(fit_intercept=False).fit(X, y)

In [ ]:
reg.coef_.flatten()

In [ ]:
mean_squared_error(y, reg.predict(X))

In [ ]:
w.flatten()

In [ ]:
mean_squared_error(y, X.dot(w))